In [11]:
import cv2
import glob
import numpy as np   
import math

In [12]:
def undistort_image(img):
    k = [[9.037596e+02, 0.000000e+00, 6.957519e+02], [0.000000e+00, 9.019653e+02, 2.242509e+02],[0.000000e+00 ,0.000000e+00, 1.000000e+00]]
    k=np.array(k)
    d = [[-3.639558e-01, 1.788651e-01, 6.029694e-04, -3.922424e-04, -5.382460e-02]]
    d=np.array(d)
    h,  w = img.shape[:2]
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(k,d,(w,h),1,(w,h))
    dst = cv2.undistort(img, k, d, None, newcameramtx)

    # crop the image
    x,y,w,h = roi
    dst = dst[y:y+h, x:x+w]
    return dst

def mean(list):
    """
    calculate mean of list
    """
    return float(sum(list)) / max(len(list), 1)


In [13]:
def detect_white_lines(img):
    lower_mask_white = np.array([0, 220, 0], dtype='uint8')
    upper_mask_white = np.array([255, 255, 255], dtype='uint8')
    mask_white = cv2.inRange(img, lower_mask_white, upper_mask_white)
    white_detect = cv2.bitwise_and(img, img, mask=mask_white).astype(np.uint8)
    return white_detect

In [23]:
# Four corners of the book in source image
pts_src = np.array([[565, 36], [724, 36], [994, 269],[178, 270]])
pts_dst = np.array([[50, 0], [250, 0], [250, 500], [0, 500]])
H_Matrix, status = cv2.findHomography(pts_src, pts_dst)


def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    
#     left_line_x = []
#     left_line_y = []
#     right_line_x = []
#     right_line_y = []
#     right_x_start =0
#     right_x_end =0
#     left_x_start =0
#     left_x_end =0
 
#     for line in lines:
#         for x1, y1, x2, y2 in line:
#             slope = (y2 - y1) / (x2 - x1)

#     fla = 0
#     flag = 0
#     if slope <= 0:
#         left_line_x.extend([x1, x2])
#         left_line_y.extend([y1, y2])
#     else:
#         right_line_x.extend([x1, x2])
#         right_line_y.extend([y1, y2])
        
#     min_y = int(img.shape[0] * (3 / 5))
#     max_y = int(img.shape[0])
    
#     if(len(left_line_y) != 0 and len(left_line_x) != 0):
#         flag = 1
#         poly_left = np.poly1d(np.polyfit(left_line_y,left_line_x,deg=1))
        
#     if(flag==1):
#         left_x_start = int(poly_left(max_y))
#         left_x_end = int(poly_left(min_y))
    
#     if(len(right_line_y) != 0 and len(right_line_x) != 0):
#         fla = 1
#         poly_right = np.poly1d(np.polyfit(right_line_y,right_line_x,deg=1))
        
#     if(fla==1):
#         right_x_start = int(poly_right(max_y))
#         right_x_end = int(poly_right(min_y))
        
#     if(fla==1 and flag ==1):
#         line_image = draw_lines(line_img,[[
#                 [left_x_start, max_y, left_x_end, min_y],
#                 [right_x_start, max_y, right_x_end, min_y],
#             ]],
#             thickness=5)
  
    draw_lines(line_img, lines)
    return line_img

# def draw_lines(img, lines, color=[0, 255, 0], thickness=2):
#     x=[]
#     y=[]
#     if lines is None:
#         return
#     n=1
#     x1Avg = 0
#     x2Avg = 0
#     y1Avg = 0
#     y2Avg = 0
#     for line in lines:
#         for x1,y1,x2,y2 in line:
# #             print(x1,y1,x2,y2)
# #             x += [x1, x2]
# #             y += [y1, y2]
# #             m,b = np.polyfit(x, y, 1)
# #             cv2.line(img, (x1, y1), (x2, y2), color, thickness)
#             x1Avg = x1Avg + (x1 - x1Avg)/n
#             x2Avg = x2Avg + (x2 - x2Avg)/n
#             y1Avg = y1Avg + (y1 - y1Avg)/n
#             y2Avg = y2Avg + (y2 - y2Avg)/n
#             n += 1
#             [slope, intercept] = np.polyfit([x1Avg, x2Avg], [y1Avg, y2Avg], 1)

# def draw_lines(img, lines, color=[255, 0, 0], thickness=12):

#     bLeftValues     = []  # b of left lines
#     bRightValues    = []  # b of Right lines
#     mPositiveValues = []  # m of Left lines
#     mNegitiveValues = []  # m of Right lines
    
#     for line in lines:
#         for x1,y1,x2,y2 in line:
#             cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            
#             # calculate slope and intercept
#             m = (y2-y1)/(x2-x1)
#             b = y1 - x1*m
            
#             # threshold to check for outliers
#             if m >= 0 and (m < 0.2 or m > 0.8):
#                 continue
#             elif m < 0 and (m < -0.8 or m > -0.2):
#                 continue
                
#             # seperate positive line and negative line slopes
#             if m > 0:
#                 mPositiveValues.append(m)
#                 bLeftValues.append(b)
#             else:
#                 mNegitiveValues.append(m)
#                 bRightValues.append(b)
    
#     # Get image shape and define y region of interest value
#     imshape = img.shape
#     y_max   = imshape[0] # lines initial point at bottom of image    
#     y_min   = 330        # lines end point at top of ROI

#     # Get the mean of all the lines values
#     AvgPositiveM = mean(mPositiveValues)
#     AvgNegitiveM = mean(mNegitiveValues)
#     AvgLeftB     = mean(bLeftValues)
#     AvgRightB    = mean(bRightValues)

#     # use average slopes to generate line using ROI endpoints
#     if AvgPositiveM != 0:
#         x1_Left = (y_max - AvgLeftB)/AvgPositiveM
#         y1_Left = y_max
#         x2_Left = (y_min - AvgLeftB)/AvgPositiveM
#         y2_Left = y_min
#     if AvgNegitiveM != 0:
#         x1_Right = (y_max - AvgRightB)/AvgNegitiveM
#         y1_Right = y_max
#         x2_Right = (y_min - AvgRightB)/AvgNegitiveM
#         y2_Right = y_min

#         # define average left and right lines
#         cv2.line(img, (int(x1_Left), int(y1_Left)), (int(x2_Left), int(y2_Left)), color, thickness) #avg Left Line
#         cv2.line(img, (int(x1_Right), int(y1_Right)), (int(x2_Right), int(y2_Right)), color, thickness) #avg Right Line

def draw_lines(img, lines, color=[0, 255, 0], thickness=12):
    """
    This function draws `lines` with `color` and `thickness`.    
    """
    imshape = img.shape
    
    # these variables represent the y-axis coordinates to which 
    # the line will be extrapolated to
    ymin_global = img.shape[0]
    ymax_global = img.shape[0]
    
    # left lane line variables
    all_left_grad = []
    all_left_y = []
    all_left_x = []
    
    # right lane line variables
    all_right_grad = []
    all_right_y = []
    all_right_x = []
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            gradient, intercept = np.polyfit((x1,x2), (y1,y2), 1)
            ymin_global = min(min(y1, y2), ymin_global)
            
            if (gradient > 0):
                all_left_grad += [gradient]
                all_left_y += [y1, y2]
                all_left_x += [x1, x2]
            else:
                all_right_grad += [gradient]
                all_right_y += [y1, y2]
                all_right_x += [x1, x2]
    
    left_mean_grad = np.mean(all_left_grad)
    left_y_mean = np.mean(all_left_y)
    left_x_mean = np.mean(all_left_x)
    left_intercept = left_y_mean - (left_mean_grad * left_x_mean)
    
    right_mean_grad = np.mean(all_right_grad)
    right_y_mean = np.mean(all_right_y)
    right_x_mean = np.mean(all_right_x)
    right_intercept = right_y_mean - (right_mean_grad * right_x_mean)
    
    # Make sure we have some points in each lane line category
    if ((len(all_left_grad) > 0) and (len(all_right_grad) > 0)):
        upper_left_x = int((ymin_global - left_intercept) / left_mean_grad)
        lower_left_x = int((ymax_global - left_intercept) / left_mean_grad)
        upper_right_x = int((ymin_global - right_intercept) / right_mean_grad)
        lower_right_x = int((ymax_global - right_intercept) / right_mean_grad)

        cv2.line(img, (upper_left_x, ymin_global), 
                      (lower_left_x, ymax_global), color, thickness)
        cv2.line(img, (upper_right_x, ymin_global), 
                      (lower_right_x, ymax_global), color, thickness)
        
        pts = np.array([[upper_left_x, ymin_global],[upper_right_x, ymin_global],[lower_right_x, ymax_global],[lower_left_x, ymax_global]], np.int32)
        pts = pts.reshape((-1,1,2))
#         cv2.polylines(img,[pts],True,(0,255,255)) 
        cv2.fillConvexPoly(img, pts, (100,100,100), lineType=8, shift=0) 

            

 
for img in glob.glob(r"C:\Users\prade\OneDrive\Documents\UMD_Robotics\Courses\ENPM 673 - Perception\Project2\data_1-20200307T183325Z-001\data_1\data\*.png"):
    cv_img = cv2.imread(img)
    crop_img = cv_img[235:512, 0:1392] 
    #     dst = cv2.fastNlMeansDenoisingColored(crop_img, None, 5, 5, 7, 15) 
    nice_img = undistort_image(crop_img)
    white_img = detect_white_lines(nice_img)
    white_img = cv2.GaussianBlur(white_img,(5,5),0) 
    im_out = cv2.warpPerspective(white_img, H_Matrix, (300,600))
    edges = cv2.Canny(im_out,100,200)
    edges[0:500,220:300] = 0
    cv2.imshow("im_out",edges)
    
    rho = 2
    theta = np.pi/180
    #threshold is minimum number of intersections in a grid for candidate line to go to output
    threshold = 100
    min_line_len = 150
    max_line_gap = 800
    α=0.8
    β=1
    λ=0
    
    line_image = hough_lines(edges, rho, theta, threshold, min_line_len, max_line_gap)
    result = cv2.addWeighted(im_out, α, line_image, β, λ)
    out = cv2.warpPerspective(result, np.linalg.inv(H_Matrix), (1328,205))
    
    # # Image Addition
    rows, cols, channels = out.shape
    roi = nice_img[0:rows, 0:cols]

    # Now create a mask of logo and create its inverse mask
    img2gray = cv2.cvtColor(out, cv2.COLOR_BGR2GRAY)

    # add a threshold
    ret, mask = cv2.threshold(img2gray, 5, 255, cv2.THRESH_BINARY_INV)

    mask_inv = cv2.bitwise_not(mask)
    # Now black-out the area of logo in ROI
    img1_bg = cv2.bitwise_and(roi, roi, mask=mask)

    # Take only region of logo from logo image.
    img2_fg = cv2.bitwise_and(out, out, mask=mask_inv)

    Final = cv2.add(img1_bg, img2_fg)
    cv2.imshow("Final_output", Final)
    
    cv2.imshow("output",out)
    cv2.imshow("white_img",white_img)
    cv2.imshow("edges",edges)
    cv2.imshow("result",result)
    cv2.waitKey(1)
#     break
    
cv2.destroyAllWindows()
    

c:\users\prade\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:59: RankWarning: Polyfit may be poorly conditioned
c:\users\prade\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:59: RankWarning: Polyfit may be poorly conditioned
c:\users\prade\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:59: RankWarning: Polyfit may be poorly conditioned
c:\users\prade\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:59: RankWarning: Polyfit may be poorly conditioned
c:\users\prade\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:59: RankWarning: Polyfit may be poorly conditioned
c:\users\prade\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:59: RankWarning: Polyfit may be poorly conditioned
c:\users\prade\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:59: RankWarning: Polyfit may be poor

In [ ]:
import cv2
import numpy as np

def draw_circle(event,x,y,flags,param):
    global mouseX,mouseY
    if event == cv2.EVENT_FLAG_LBUTTON:
        cv2.circle(img,(x,y),10,(255,0,0),-1)
        mouseX,mouseY = x,y

# img = np.zeros((512,512,3), np.uint8)
img = cv2.imread('messi.png')

cv2.namedWindow('image')
cv2.setMouseCallback('image',draw_circle)

while(1):
    cv2.imshow('image',img)
    k = cv2.waitKey(20) & 0xFF
    if k == 27:
        
        
        
        

        break
    elif k == ord('a'):
        print (mouseX,mouseY)

565 36
724 35
994 269
178 270
